In [1]:
!pip install fuzzywuzzy

In [2]:
import re
import requests
from fuzzywuzzy import fuzz
from urllib.parse import urlparse

# List of common keywords and patterns used in smishing
MALICIOUS_PATTERNS = {
    "Urgency & Scam": r"urgent",  # Common term used to create urgency
    "Verify Account": r"verify your account",  # Common phrase in phishing attempts
    "Financial Smishing": r"bank account",  # Changed to Financial Smishing
    "Security Code": r"security code",  # Commonly used to trick users into revealing codes
    "Login Credentials": r"login",  # Often used in phishing to steal credentials
    "Personal Info Request": r"personal information",  # Sensitive data requested
    "Click Here": r"click here",  # Often used to lure users into clicking malicious links
    "Prize & Rewards": r"won",  # Often used in scams claiming winnings
    "Prize Scam": r"prize",  # Often used in fake prize schemes
    "Congratulations": r"congratulations",  # Often used to trick users into thinking they’ve won something
    "Update Details": r"update your details",  # Often used to prompt users to update information fraudulently
    "Secure Account": r"secure your account",  # Common in phishing to make it seem urgent
    "Verify Identity": r"verify your identity",  # Commonly used to trick users into revealing personal info
    "URL": r"http[s]?://[^\s]+",  # Detect URLs
    "Phone Number": r"\+?\d{1,4}[-.\s]?\(?\d{1,3}\)?[-.\s]?\d{1,3}[-.\s]?\d{1,4}",  # Detect phone numbers
    "Email": r"[a-zA-Z0-9._%+-]+@[a-zAZ0-9.-]+\.[a-zA-Z]{2,}",  # Detect email addresses
}

# List of blacklisted domains (example, can be extended)
BLACKLISTED_DOMAINS = ["bit.ly", "short.ly", "tinyurl.com", "malicious.com"]

def is_blacklisted_url(url):
    """
    Checks if a URL belongs to a blacklisted domain.

    Args:
        url (str): The URL to be checked.

    Returns:
        bool: True if the URL belongs to a blacklisted domain, False otherwise.
    """
    try:
        domain = urlparse(url).netloc
        return domain in BLACKLISTED_DOMAINS
    except Exception:
        return False

def is_smishing_message(message):
    """
    Checks if the message contains patterns indicative of smishing.

    Args:
        message (str): The SMS message to be checked.

    Returns:
        dict: A dictionary with keys as detected pattern categories and values as a list of matched patterns.
    """
    detected_patterns = {key: [] for key in MALICIOUS_PATTERNS}  # Initialize empty lists for all categories

    for category, pattern in MALICIOUS_PATTERNS.items():
        matches = re.findall(pattern, message, re.IGNORECASE)
        if matches:
            if category == "URL":
                # Check for suspicious URLs
                for url in matches:
                    if is_blacklisted_url(url):
                        detected_patterns["URL"].append(url)
            elif category == "Phone Number" or category == "Email":
                detected_patterns["Phone Number"].extend(matches)  # Add to respective lists
                detected_patterns["Email"].extend(matches)
            else:
                # For other patterns like urgency, phishing attempts, etc.
                detected_patterns[category].extend(matches)

    return detected_patterns

def fuzzy_match(message, target_phrase, threshold=80):
    """
    Checks for fuzzy matching (misspelled versions of keywords) using Levenshtein distance.

    Args:
        message (str): The message to check.
        target_phrase (str): The phrase to match.
        threshold (int): The minimum similarity score for a match (default is 80%).

    Returns:
        bool: True if a fuzzy match is found, False otherwise.
    """
    return fuzz.partial_ratio(message.lower(), target_phrase.lower()) >= threshold

# Example usage
if __name__ == "__main__":
    sms_message = input("Enter the SMS message to check: ")

    # Check for smishing patterns and fuzzy matches
    detected = is_smishing_message(sms_message)

    # Check for fuzzy matches with some common patterns
    for key, value in MALICIOUS_PATTERNS.items():
        if key not in ["URL", "Phone Number", "Email"]:  # Skip URL, phone, and email pattern matching
            if fuzzy_match(sms_message, value):
                detected["Urgency & Scam"].append(f"Fuzzy match found for pattern: {key}")

    # Output results
    if any(detected.values()):
        print("Warning: This message may be a smishing attempt. Detected patterns:")
        for category, matches in detected.items():
            if matches:
                smishing_category = category.replace("Phishing", "Smishing")  # Replace 'Phishing' with 'Smishing'
                print(f"{smishing_category}: {', '.join(matches)}")
    else:
        print("The message appears to be safe.")


/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Enter the SMS message to check: Congratulations! You have won a prize of $1000! Click here to claim your prize now! Secure your bank account details by verifying your identity.
Urgency & Scam: Fuzzy match found for pattern: Financial Smishing, Fuzzy match found for pattern: Click Here, Fuzzy match found for pattern: Prize & Rewards, Fuzzy match found for pattern: Prize Scam, Fuzzy match found for pattern: Congratulations, Fuzzy match found for pattern: Verify Identity
Financial Smishing: bank account
Click Here: Click here
Prize & Rewards: won
Prize Scam: prize, prize
Congratulations: Congratulations
Phone Number: 1000
Email: 1000
